In [1]:
from pyspark.sql import SparkSession
import pyspark as ps
import pyspark.sql.functions as F
from pyspark.sql.functions import col, lit, when, desc, asc
from pyspark.sql import types as T
import pandas as pd

In [2]:
#Exercício 1
df=pd.read_csv('realtor-data.zip.csv')


In [3]:
#Exercício 2
df.to_parquet('realtor-data.parquet')

In [4]:
#abrindo sessão
spark=(SparkSession
       .builder
       .appName("Python Spark SQL basic example") 
        .config("spark.some.config.option", "some-value") 
        .getOrCreate())
conf = ps.SparkConf().setMaster("yarn-client").setAppName("sparK-mer")
conf.set("spark.executor.heartbeatInterval","3600s")

In [5]:
casas=spark.read.parquet('realtor-data.parquet')
casas.printSchema()

root
 |-- brokered_by: double (nullable = true)
 |-- status: string (nullable = true)
 |-- price: double (nullable = true)
 |-- bed: double (nullable = true)
 |-- bath: double (nullable = true)
 |-- acre_lot: double (nullable = true)
 |-- street: double (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip_code: double (nullable = true)
 |-- house_size: double (nullable = true)
 |-- prev_sold_date: string (nullable = true)



In [ ]:
#Exercício 3

casas_part=casas.repartition(32)
casas_part.cache() 

DataFrame[brokered_by: double, status: string, price: double, bed: double, bath: double, acre_lot: double, street: double, city: string, state: string, zip_code: double, house_size: double, prev_sold_date: string]

In [7]:
casas_part.rdd.getNumPartitions()

32

In [8]:
#Exercício 4
casas_part.groupBy('state').pivot('status').count().orderBy('state').show()

+--------------------+--------+--------------+------+
|               state|for_sale|ready_to_build|  sold|
+--------------------+--------+--------------+------+
|                NULL|       2|             6|  NULL|
|             Alabama|   26486|           802|  6765|
|              Alaska|    2581|          NULL|  NULL|
|             Arizona|   37657|          1042| 34126|
|            Arkansas|   19013|             7|  4025|
|          California|  101034|          1168|125013|
|            Colorado|   26911|           706|  4676|
|         Connecticut|   13940|             4|    64|
|            Delaware|    5143|           198|  3287|
|District of Columbia|    2922|          NULL|  3703|
|             Florida|  161216|          2624| 85592|
|             Georgia|   51517|           517| 28943|
|                Guam|     489|          NULL|  NULL|
|              Hawaii|    6994|          NULL|   249|
|               Idaho|   11899|           236|  4625|
|            Illinois|   531

In [9]:
#Exercício 5
casas_part.select('city','house_size').orderBy(F.desc('house_size')).show(5)

+-----------+-----------+
|       city| house_size|
+-----------+-----------+
|      Tacna|1.0404004E9|
|     Warden|  1.29922E7|
|    Pierson|  9842382.0|
|Rocksprings|  7971480.0|
|  Wenatchee|  3484800.0|
+-----------+-----------+
only showing top 5 rows



In [10]:
#Exercício 6
tabela=pd.DataFrame({'state':['Connecticut', 'Puerto Rico', 'Massachusetts', 'New York'], 
              'population':[3606, 3264, 6985, 8468]})
tabela.to_parquet('tabela.parquet')
tabela=spark.read.parquet('tabela.parquet')
tabela.show()

+-------------+----------+
|        state|population|
+-------------+----------+
|  Connecticut|      3606|
|  Puerto Rico|      3264|
|Massachusetts|      6985|
|     New York|      8468|
+-------------+----------+



In [11]:
casas_join=casas_part.join(tabela,'state', 'inner')
casas_join.show()


+-------------+-----------+--------+---------+----+----+--------+---------+-------------+--------+----------+--------------+----------+
|        state|brokered_by|  status|    price| bed|bath|acre_lot|   street|         city|zip_code|house_size|prev_sold_date|population|
+-------------+-----------+--------+---------+----+----+--------+---------+-------------+--------+----------+--------------+----------+
|Massachusetts|    24035.0|for_sale| 399000.0| 6.0| 3.0|    0.07|1105804.0|      Peabody|  1960.0|    2547.0|    1994-08-29|      6985|
|Massachusetts|    21437.0|for_sale| 800000.0| 4.0| 2.0|    0.08| 388344.0|       Revere|  2151.0|    1900.0|          NULL|      6985|
|     New York|    40690.0|for_sale| 262500.0| 3.0| 1.0|    0.27| 560703.0|         Troy| 12180.0|    1700.0|    2019-08-02|      8468|
|Massachusetts|    22731.0|for_sale| 265000.0| 1.0| 1.0|    NULL|1761878.0|     Weymouth|  2191.0|     772.0|    2001-06-29|      6985|
|     New York|    91607.0|for_sale| 549000.0| 5

In [12]:
print('original: ', casas_part.count())
print('inner join: ', casas_join.count())

original:  2226382
inner join:  158334
